In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import os
import random

In [45]:
driver=None

pagenumber=1

URL=None

def initiate():
    
    global driver
    
    driver=wd.Firefox(executable_path='C:/geckodriver.exe')
    
    driver.maximize_window()
    
    driver.get('https://www.helium10.com/blog/')
    
    sleep(4)

In [3]:
def change_language():
    
    global driver
    
    arrow=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[3]')

    arrow.click()

    swedish=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/div/div[2]/div[2]/div[2]/div/div[2]/div[93]')

    swedish.click()
        

In [4]:
def extract_blog():
    
    global driver
    
    text=driver.find_element_by_xpath('//div[@class="article-content"]').text
    
    return text

In [46]:
def google_translate():
    
    global driver
    
    driver.execute_script('window.open("");')
    
    handles=driver.window_handles
    
    driver.switch_to.window(handles[-1])
    
    sleep(random.randint(40,55))
    
    driver.get('https://translate.google.co.in/')
    sleep(2)
    
    doc=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[1]/div/div[2]')
    doc.click()
    sleep(1)
    
    inp=driver.find_element_by_xpath('//*[@id="tlid-file-input"]')
    inp.send_keys(os.path.abspath('Helium10/Page.txt'))
    
    sleep(3)
    
    change_language()
    
    translate=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[2]/div/form/div[2]/div[2]/input')
    translate.click()
    
    sleep(3)
    
    translated_text=driver.find_element_by_tag_name('body').text
        
    driver.close()
    
    driver.switch_to.window(handles[0])
    
    delete_file()
    
    return translated_text
    

In [6]:
def write_to_file(text):
    
    global URL
    
    global pagenumber
    
    file_name='Helium10/Page' + str(pagenumber) + '.txt'
    
    pagenumber+=1
    
    with open(file_name,'a') as file:
        
        file.write('URL:{}'.format(URL))
        
        file.write('\n\n')
        
        for i in text:
            
            try:
                
                file.write(i)
                
            except:
                
                pass
        
        file.write('\n\n\n\n')

In [7]:
def create_temp_file(text):
    
    with open('Helium10/Page.txt','w') as file:
        
        for i in text:
            
            try:
        
                file.write(i)
            
            except:
                
                pass
    

In [8]:
def delete_file():
    
    try:
        
        os.remove('Helium10/Page.txt')
        
        print('File deleted successfully !')
        
    except:
        
        pass

In [9]:
def next_blog():
    
    global driver
    
    global URL
    
    tags=driver.find_elements_by_xpath('//div[@class="blog-post"]')
    
    for tag in tags:
        
        link=tag.find_element_by_tag_name('a').get_attribute('href')
        
        driver.execute_script('window.open("");')
        
        handles=driver.window_handles
        
        driver.switch_to.window(handles[-1])
        
        driver.get(link)
        
        URL=link
        
        text=extract_blog()
        
        create_temp_file(text)
        
        driver.close()
        
        driver.switch_to.window(handles[0])
        
        translated=google_translate()
        
        write_to_file(translated)
        
        delete_file()
            

In [40]:
def next_page():
    
    global driver
    
    found=False
    
    try:
        
        next_=driver.find_element_by_partial_link_text('Next')
        
        body=driver.find_element_by_tag_name('body')
        
        for i in range(55):
            
            body.send_keys(Keys.ARROW_DOWN)
            
        sleep(3)
        
        next_.click()
        
        found=True
        
        return found
            
    except:
        
        return False
        
        pass
    

In [11]:
def create_directory():
    
    try:
        
        os.mkdir('Helium10')
        
    except:
        
        pass

In [12]:
def executor():
    
    create_directory()
    
    initiate()
    
    next_blog()
    
    while next_page():
        
        next_blog()
        

In [44]:
executor()